In [1]:
%load_ext autoreload

%autoreload 2

from medcat.cat import CAT
from medcat.utils.vocab import Vocab
from medcat.cdb import CDB
from medcat.prepare_cdb import PrepareCDB

from icd9cms import search
import os
import pandas as pd
from collections import defaultdict
import json
from matplotlib import pyplot as plt

In [2]:
codes = pd.read_csv('data/admission_codes.csv')
df = codes.groupby('icd9_code').count()
df.reset_index(inplace=True)
df.columns = ['cui', 'count', '_', '_', '_']
clean_codes = df.loc[:, ['cui', 'count']]

### Top 5 Occurring Codes in MIMIC-III

In [3]:
clean_codes.sort_values('count', ascending=False).head(5)

,cui,count
1920,4019,20703
2066,4280,13111
2055,42731,12891
1977,41401,12429
2897,5849,9119


In [4]:
# needed columns 
# cui,str,onto,tty,tui,sty,desc,is_unique,examples
code_meta_dir = defaultdict(list)
for row in clean_codes.itertuples():
    found_code = search(row.cui)
    code_meta_dir['cui'].append(row.cui)
    code_meta_dir['count'].append(row.count)
    code_meta_dir['str'].append(found_code.short_desc)
    code_meta_dir['onto'].append('icd9')
    parent_code = found_code
    if  found_code.code[0] != 'E' and len(found_code.code) > 3:
        parent_code = search(found_code.code[0:3])
    elif found_code.code[0] == 'E' and len(found_code.code) > 4:
        parent_code = search(found_code.code[0:4])
    code_meta_dir['tui'].append(parent_code.code)
    code_meta_dir['sty'].append(parent_code.short_desc)
    code_meta_dir['desc'].append(found_code.long_desc or found_code.short_desc)
    code_meta_dir['is_unique'].append(-1)

In [5]:
cdb_df = pd.DataFrame(code_meta_dir).sort_values('count', ascending=False)

In [6]:
cdb_df.sort_values('count', ascending=False).head(10)

,cui,count,str,onto,tui,sty,desc,is_unique
1920,4019,20703,Hypertension NOS,icd9,401,Essential hypertension,Unspecified essential hypertension,-1
2066,4280,13111,CHF NOS,icd9,428,Heart failure,"Congestive heart failure, unspecified",-1
2055,42731,12891,Atrial fibrillation,icd9,427,Cardiac dysrhythmias,Atrial fibrillation,-1
1977,41401,12429,Crnry athrscl natve vssl,icd9,414,Other forms of chronic ischemic heart disease,Coronary atherosclerosis of native coronary ar...,-1
2897,5849,9119,Acute kidney failure NOS,icd9,584,Acute kidney failure,"Acute kidney failure, unspecified",-1
811,25000,9058,DMII wo cmp nt st uncntr,icd9,250,Diabetes mellitus,Diabetes mellitus without mention of complicat...,-1
931,2724,8690,Hyperlipidemia NEC/NOS,icd9,272,Disorders of lipoid metabolism,Other and unspecified hyperlipidemia,-1
2484,51881,7497,Acute respiratry failure,icd9,518,Other diseases of lung,Acute respiratory failure,-1
2965,5990,6555,Urin tract infection NOS,icd9,599,Other disorders of urethra and urinary tract,"Urinary tract infection, site not specified",-1
2576,53081,6326,Esophageal reflux,icd9,530,Diseases of esophagus,Esophageal reflux,-1


## Link Pre-trained UMLS to ICD-9-CM
CSV mapping file from: https://bioportal.bioontology.org/ontologies/ICD9CM/?p=summary

After testing with ICD9 mapping. Many mappings appear to be missing with UMLS. Using the trained / built ICD10 to UMLS mapping

In [7]:
# mapping = pd.read_csv('ICD9CM.csv')

## Mapping ICD9 to ICD10

In [65]:
codes = pd.read_csv('data/admission_codes.csv')

In [9]:
icd9_to_10_mapping = pd.read_csv('refdata/icd9toicd10cmgem.csv')
sorted_codes = codes.icd9_code.value_counts().reset_index().sort_values('icd9_code', ascending=False)
sorted_codes.columns = ['code', 'count']
count = 0
icd10 = []
for c in sorted_codes.code:
    mapped_rows = icd9_to_10_mapping[icd9_to_10_mapping.icd9cm == c]   
    icd10.append(','.join(mapped_rows.icd10cm.tolist()) if 
                 mapped_rows.shape[0] > 1 else mapped_rows.icd10cm.iloc[0])
sorted_codes['icd10'] = icd10

In [10]:
total_assigned_codes = sorted_codes['count'].sum()

In [11]:
sorted_codes['count'].iloc[:300].sum()

450732

## Code Proportions of Total Assigned Codes
We go with thea top 400 ICD-9 codes in the following analysis, giving  is a reasonable number for 'majority' coverage of the codes in MIMIC-III without validating all ~6k. 

In [12]:
code_ranges = [50, 100, 200, 300, 400, 500, 600]
props_of_total_codes = [(r, round((sorted_codes['count'].iloc[:r].sum() / total_assigned_codes) * 100, 2)) for r in code_ranges] 

In [13]:
props_of_total_codes

[(50, 38.24),
 (100, 50.11),
 (200, 63.39),
 (300, 71.01),
 (400, 76.2),
 (500, 79.96),
 (600, 82.9)]

## Filter Down to top 400 ICD-9 Codes
That have first been mapped to ICD-10 codes, then mapped to UMLS CUI Codes.

In [14]:
icd10_mapping = pd.read_csv('refdata/UMLS_to_ICD10.csv')

In [15]:
icd10_mapping = icd10_mapping.loc[:, ['Class ID', 'CUI', 'Semantic type UMLS property']]

In [16]:
icd10_mapping['tuis'] = icd10_mapping['Semantic type UMLS property']\
    .str.split('|').dropna().apply(\
        lambda tui_list: ','.join([l.split('/')[-1] for l in tui_list]))

In [17]:
icd10_mapping['code'] = icd10_mapping['Class ID'].str.split('/').apply(lambda l: l[-1])

In [18]:
from itertools import chain
icd10_codes_to_keep = list(chain.from_iterable(sorted_codes[0:400].icd10.str.split(',').tolist()))

In [19]:
icd10_mapping['code'] = icd10_mapping.code.str.replace('.', '')
tuis_to_keep = chain.from_iterable([icd10_mapping[icd10_mapping.code == code].tuis.tolist() for code in icd10_codes_to_keep])
tuis_to_keep = set(chain.from_iterable([tui.split(',') for tui in tuis_to_keep]))
cuis_to_keep = chain.from_iterable([icd10_mapping[icd10_mapping.code == code].CUI.tolist() for code in icd10_codes_to_keep])
cuis_to_keep = list(chain.from_iterable([cui.split('|') for cui in cuis_to_keep]))
cuis_to_keep = list(set(cuis_to_keep))

In [20]:
icd10_mapping = pd.read_csv('refdata/UMLS_to_ICD10.csv')
icd10_mapping['icd10'] = icd10_mapping['Class ID'].str.split('/').apply(lambda l: l[-1])
icd10_mapping['icd10'] = icd10_mapping.icd10.str.replace('.', '')

## Code Reconciliation
Given a set of predicted UMLS concepts for the DD subsections made by MedCAT, we aim to reconcile the codes assigned to the discharge diagnosis subsections in MIMIC-III.

Firstly, we map the predicted codes back into their ICD-9 counterparts. MedCAT already contains a UMLS to ICD-10-CM mapping, so we directly use that mapping supplied by the tool.

In [21]:
notes = pd.read_csv('data/mimic_dd_icd9_sections.csv')
notes['name'] = notes.name.str.split(':').apply(lambda l: l[1])
codes = pd.read_csv('data/admission_codes.csv').drop('Unnamed: 0', axis=1)
objs = json.load(open('data/mimic_dd_annos.json'))

### Map parsed UMLS concepts with attached ICD-10 codes to ICD9

In [ ]:
from reconcile import map_medcat_annos_to_icd9

In [ ]:
parsed_dd_mapped_to_icd9_from_intellij = map_medcat_annos_to_icd9(objs, notes, codes, cuis_to_keep)

### Reconcile Codes
As UMLS codes can map many-to-many to ICD-10 and ICD-9 codes, a single UMLS prediction by MedCAT can map to many associated ICD codes. If the MIMIC-III assigned codes matches one of these UMLS assigned codes we assume the MIMIC-III assigned code is correct and remove all others that refer to the same span of text.

Then split the reconciled codes into 3 datasets:
- Assigned in MIMIC-III and Not Predicted by MedCAT, denoted **A_NP** in the paper.
- Predicted by MedCAT and Assigned in MIMIC-III, denoted **P_A** in the paper. 
- Predicted by MedCAT and Not Assigned in MIMIC-III, denoted **P_NA** in the paper. 

In [50]:
from reconcile import remove_duplicate_mapped_assigned_codes

In [106]:
joined_df = remove_duplicate_mapped_assigned_codes(notes, codes, parsed_dd_mapped_to_icd9_from_intellij)

In [23]:
# all_rows = []
# for note, ents in zip(notes.itertuples(), objs):
#     for e in ents['entities']:
#         if e['cui'] in cuis_to_keep:
#             e['hadm'] = note.name
#             all_rows.append(e)
# df = pd.DataFrame(all_rows)
# sorted_top_400 = codes.groupby('icd9_code').count().loc[:, 'hadm_id'].sort_values(ascending=False)[0:400].index.tolist()
# codes = codes[codes.icd9_code.isin(sorted_top_400)]
# codes = codes[codes.hadm_id.isin(notes.name)]

# icd9_to_10_mapping = pd.read_csv('refdata/icd9toicd10cmgem.csv')
# icd9_to_10_mapping = icd9_to_10_mapping[icd9_to_10_mapping.icd9cm.isin(sorted_top_400)]
# filtered_icd_10_codes = icd9_to_10_mapping.icd10cm.tolist()
# 
# parsed_dd_mapped_to_icd9 = []
# no_mapping_found = []
# for r in df.itertuples():
#     for code in r.icd10.split(','):
#         clean_code = code.replace('.', '')
#         if clean_code in filtered_icd_10_codes:
#             mapped_icd9_codes = icd9_to_10_mapping[icd9_to_10_mapping.icd10cm == clean_code].icd9cm.tolist()
#             mapped_icd9_code_instances = [{
#                 'acc': r.acc,
#                 'cui': r.cui,
#                 'hadm': r.hadm,
#                 'pretty_name': r.pretty_name,
#                 'source_value': r.source_value,
#                 'start': r.start,
#                 'end': r.end,
#                 'icd9': icd9_code,
#                 'icd10': code
#             } for icd9_code in mapped_icd9_codes]
#             parsed_dd_mapped_to_icd9 += mapped_icd9_code_instances
#         else:
#             no_mapping_found.append(code)
# parsed_dd_mapped_to_icd9 = pd.DataFrame(parsed_dd_mapped_to_icd9)

# codes = codes.reset_index(drop=True)
# codes.columns = ['hadm', 'icd9', 'short_title', 'long_title']

# joined_df = []
# for h_key, codes_by_hadm in codes.groupby('hadm'):
#     pred_codes = parsed_dd_mapped_to_icd9[parsed_dd_mapped_to_icd9.hadm == str(h_key)]
#     rows_to_keep = []
#     mult_starts = []
#     matched_icd9_codes = []
#     for code_row in codes_by_hadm.itertuples():
#         if code_row.icd9 in pred_codes.icd9.tolist():
#             # true positive rows. De-dupe is needed.
#             pred_code_match = pred_codes[pred_codes.icd9 == code_row.icd9].iloc[0]
#             pred_code_match = pred_code_match.to_dict()
#             pred_code_match['match'] = 'match'
#             matched_icd9_codes.append(pred_code_match['icd9'])
#             rows_to_keep.append(pred_code_match)
#             mult_starts.append(pred_code_match['start'])
#     no_match_pred_codes = pred_codes[~pred_codes.start.isin(mult_starts)]
#     no_match_pred_codes = no_match_pred_codes.to_dict(orient='records')
#     for p in no_match_pred_codes:
#         p['match'] = 'pred_no_assign'
#     rows_to_keep.extend(no_match_pred_codes)
    
#     no_match_codes = codes_by_hadm[~codes_by_hadm.icd9.isin(matched_icd9_codes)].loc[:, ['hadm', 'icd9']]
#     no_match_codes = no_match_codes.to_dict(orient='records')
#     for p in no_match_codes:
#         p['match'] = 'assigned_no_pred'
#     rows_to_keep.extend(no_match_codes)
#     joined_df.extend(rows_to_keep)

### Plot Base Predictions by MedCAT of DD Susections
- Predicted codes that match with Assigned codes from MIMIC-III
- Predicted codes but have no matching assigned codes. (These are the most interesting cases, where we can begin to argue where the coding is unreliable).
- We accept the gap between assigned codes and no predictions is large due to:
 - codes coming from outside the DD subsection
 - possibly overcoding

In [41]:
joined_df = pd.DataFrame(joined_df)
matched_codes = joined_df[joined_df.match == 'match'].icd9.unique()
joined_df_assignedno_pred = joined_df[joined_df.match == 'assigned_no_pred']
joined_df_assignedno_pred = joined_df_assignedno_pred[joined_df_assignedno_pred.icd9.isin(matched_codes)].shape[0]
joined_df_assignedpred_no_assign = joined_df[joined_df.match == 'pred_no_assign']
joined_df_assignedpred_no_assign = joined_df_assignedpred_no_assign[joined_df_assignedpred_no_assign.icd9.isin(matched_codes)].shape[0]

In [42]:
y = [joined_df[joined_df.match != 'pred_no_assign'].shape[0],  # raw code counts
     joined_df_assignedno_pred, # raw code counts not including the codes that had at least one match.
     joined_df[joined_df.match == 'match'].shape[0], # raw code counts that match with assigned codes
     joined_df_assignedpred_no_assign # raw code counts that were not assigned but were predicted.
    ]

In [415]:
labels = ['All assign.', 'Assign., filtered', 'Matches', 'Predictions, no assign.,\nfiltered']
fig, ax = plt.subplots(figsize=(8, 5))
ax.bar(range(len(y)), y)
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)
ax.set_ylabel('# of codes assigned')
plt.savefig('./plots/code_counts_unsupervised.png')
plt.show()

## MedCAT Fine-Tuning / Validation Stratified Sampling 
We sub-sample each of the three datasets by:
 - by ICD-9 Code
 - by HADMD ID

The **A_NP** dataset is used for MedCAT 'fine-tuning', so the model can include additional synoyms or abbreviations for codes.

**P_A** and **P_NA** are used to validate the tool ensuring the model predictions are valid.

In [429]:
random_state = 42
def sample_dataset(dataset, sample_size, num_discard_lead_samples=0, only_sample_codes=None):
    samples = []
    for code, group_df in dataset.groupby('icd9'):
        if only_sample_codes is not None and code not in only_sample_codes:
            continue
        sample_df = group_df.sample(min(group_df.shape[0], sample_size), random_state=random_state).loc[:, ['hadm']]
        sample_df.iloc[num_discard_lead_samples:]
        sample_df['icd9'] = code
        sample_df['text'] = [notes[notes.name == str(sample.hadm)].text.iloc[0] for sample in sample_df.itertuples()]
        samples.append(sample_df)
    return pd.concat(samples).reset_index(drop=True)

### Sample - Prediction and Assigned

In [430]:
sample_dataset(joined_df[joined_df.match == 'match'], 10).to_csv('sampled_correct_preds.csv', index=False)

### Sample - Pred and Not Assigned

In [ ]:
sample_dataset(joined_df[joined_df.match == 'pred_no_assign'], 10).to_csv('sampled_preds_no_assign.csv', index=False)

### Sample - Assigned, No Pred

In [ ]:
sample_dataset(joined_df[joined_df.match == 'assigned_no_pred'], 10).to_csv('sampled_assigned_no_pred.csv', index=False)

# Create Datasets to be evaludated in MedCATTrainer
We use a running instance of the [MedCATTrainer](https://github.com/Cogstack/MedCATTrainer) interface.

We use the ReST API for creating datasets / projects

In [10]:
import requests
import json
import pandas as pd

### First 'Login' to the API with Username and Password.
The returned API token once logged in is used in subsequent request headers.

In [16]:
headers = {
    'Authorization': f'Token {json.loads(requests.post("http://localhost:8001/api/api-token-auth/", json={"username": "admin", "password": "admin"}).text)["token"]}',
}

### Create Datasets for Fine-Tuning / Validation
Each ICD-9 Code for each of the 3 (**A_NP**, **P_A**, **P_NA**) are created.

In [100]:
def create_datasets(filename, dataset_name_prefix, dataset_desc_prefix):
    dataset_ids = []
    for code, group_df in list(pd.read_csv(filename).groupby('icd9')):
        group_df.columns = ['name', 'icd9', 'text']
        payload = {
            'dataset_name': f'{dataset_name_prefix}{code}',
            'dataset': group_df.loc[:, ['name', 'text']].to_dict(),
            'description': f'{dataset_desc_prefix}{code}',
        }
        resp = requests.post('http://localhost:8001/api/create-dataset/', json=payload, headers=headers)
        dataset_ids.append(json.loads(resp.text)['dataset_id'])
    return dataset_ids

In [ ]:
assn_no_pred_dataset_ids = create_datasets('sampled_assigned_no_pred.csv', 'Assn No Pred ', 'Assigned with no prediction for code ')

In [114]:
assn_pred_dataset_ids = create_datasets('sampled_correct_preds.csv', 'Corr No Pred ', 'Assigned and predicted for code ')

In [106]:
preds_no_assgn_dataset_ids = create_datasets('sampled_preds_no_assign.csv', 'Preds No Assn ', 'Predicted but no assignment for code ')

### Create Projects for Fine-Tuning / Validation
Each ICD-9 Code for each of the 3 (**A_NP**, **P_A**, **P_NA**) are created.

We create an associated .csv file for each of 3 datasets to track the manual validation / fine-tuning effort.

In [102]:
icd10_mapping.icd10_code = icd10_mapping.icd10_code.str.replace('.', '')
icd9_to_10_mapping.reset_index(inplace=True)

In [112]:
def create_projects(project_prefix, filename, dataset_ids):
    code_maps = {'icd9_code': [], 'icd9_long_title': [], 'icd10_code': [], 'umls_cuis': []}
    for dataset_id, (code, group_df) in zip(dataset_ids, pd.read_csv(filename).groupby('icd9')):
        icd10_mapped_codes = icd9_to_10_mapping[icd9_to_10_mapping.icd9cm == code].icd10cm.tolist()
        cuis = list(chain.from_iterable([c.split('|') for c in icd10_mapping[icd10_mapping.icd10_code.isin(icd10_mapped_codes)].CUI.tolist()]))
        payload = {
            'name': f'{project_prefix} - DS: {dataset_id} code: {code} desc: {codes[codes.icd9 == code].iloc[0].short_title}',
            'description': '',
            'cuis': ','.join(cuis),
            'tuis': '',
            'require_entity_validation': True,
            'train_model_on_submit': True,
            'dataset': dataset_id,
            'concept_db': 3, # UMLS Full CDB 
            'vocab': 1, # vocab full
            'members': [1]
        }
        code_maps['icd9_code'].append(code)
        code_maps['icd9_long_title'].append(search(code).long_desc)
        code_maps['icd10_code'].append(icd10_mapped_codes)
        code_maps['umls_cuis'].append(cuis)
        resp = requests.post('http://localhost:8001/api/project-annotate-entities/', json=payload, headers=headers)
        if resp.ok:
            print(f'created project for dataset:{dataset_id}')
        else:
            print(f'error creating project for dataset:{dataset_id}')
            print(resp.text)
            raise Exception()
    return code_maps

In [207]:
df = list(pd.read_csv('sampled_assigned_no_pred.csv').groupby('icd9'))
icd9_to_10_mapping[icd9_to_10_mapping.icd9cm == '00845'].icd10cm.tolist()
df = list(pd.read_csv('sampled_correct_preds.csv').groupby('icd9'))

In [252]:
code_maps_asssn_no_preds = create_projects('Assn No Prd', 'sampled_assigned_no_pred.csv', assn_no_pred_dataset_ids)
pd.DataFrame(code_maps_asssn_no_preds).to_csv('code_maps_sampled_assgned_no_pred.csv', index=False)

In [ ]:
code_maps_preds_no_assign = create_projects('Prd N Asn', 'sampled_preds_no_assign.csv', preds_no_assgn_dataset_ids)
pd.DataFrame(code_maps_preds_no_assign).to_csv('code_maps_sampeld_preds_no_assn.csv', index=False)

In [124]:
code_maps_correct_preds = create_projects('Cor Prd', 'sampled_correct_preds.csv', assn_pred_dataset_ids)
pd.DataFrame(code_maps_correct_preds).to_csv('code_maps_sampled_correct_preds.csv', index=False)

## Manual validation of above samples of each Dataset
Each csv dataset has an associated .xlsx, with further columns added for validation.

### Further Samples of Assigned but no prediction from MedCAT
- During validation of the 'Assigned with no prediction of code', we annotate if more samples are required.
- Re-sample only those with 'TRUE' in the column of more_samples_required.

In [422]:
code_maps_assn_no_preds_complete = pd.read_excel('code_maps_sampled_assgned_no_pred.xlsx')

In [425]:
icd9_codes_to_resample = code_maps_assn_no_preds_complete[code_maps_assn_no_preds_complete.more_samples_required].icd9_code.tolist()

In [433]:
sample_dataset(joined_df[joined_df.match == 'assigned_no_pred'], 10, 10, icd9_codes_to_resample).to_csv('sampled_assigned_no_pred_more.csv', index=False)

## Load Fine-Tuned MedCAT Model, Re-Reun on DD Subsections, then Re-run Analysis


## Analyse new annotations

In [125]:
code_maps_assn_no_preds_complete = pd.read_excel('code_maps_sampled_assgned_no_pred.xlsx')

In [127]:
more_samples_required = code_maps_assn_no_preds_complete[code_maps_assn_no_preds_complete.more_samples_required == True]

In [98]:
code_maps_assn_no_preds_complete[pd.isna(code_maps_assn_no_preds_complete.Notes)]

,icd9_code,icd9_long_title,icd10_code,umls_cuis,more_samples_required,Notes,Conflict _nums,Conflicts
6,4104,Streptococcus infection in conditions classifi...,['B952'],['C2833796'],0.0,NaN,NaN,NaN
8,4119,Staphylococcus infection in conditions classif...,['B957'],['C2833801'],1.0,NaN,NaN,NaN
11,4185,Other specified bacterial infections in condit...,['B9689'],['C2833813'],0.0,NaN,"ecoli uti, ecoli pneumonia",Urosepsis
13,7032,Chronic viral hepatitis B without mention of h...,['B181'],"['C0276610', 'C0524909', 'C2911652']",0.0,NaN,NaN,NaN
15,7070,Unspecified viral hepatitis C without hepatic ...,['B1920'],"['C0019196', 'C1456263']",0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
395,V5865,Long-term (current) use of steroids,"['Z7951', 'Z7952']","['C2911186', 'C2911187']",0.0,NaN,NaN,not mentioned
396,V5867,Long-term (current) use of insulin,['Z794'],['C1455979'],0.0,NaN,NaN,not mentioned
397,V667,Encounter for palliative care,['Z515'],['C0375892'],0.0,NaN,NaN,NaN
398,V707,Examination of participant in clinical trial,['Z006'],"['C2910462', 'C3264055']",0.0,NaN,NaN,NaN


In [164]:
(code_maps_assn_no_preds_complete[code_maps_assn_no_preds_complete.more_samples_required].shape[0] / 400) * 100

46.5

In [ ]:
code_maps_assn_no_preds_complete

In [172]:
(code_maps_assn_no_preds_complete[~pd.isna(code_maps_assn_no_preds_complete.Notes)].shape[0] / 400) * 100

97.0

In [132]:
code_maps_pred_no_assn_complete = pd.read_excel('code_maps_sampeld_preds_no_assn.xlsx')
code_maps_correct_preds_complete = pd.read_excel('code_maps_sampled_correct_preds.xlsx')

In [149]:
num_good = code_maps_pred_no_assn_complete[code_maps_pred_no_assn_complete.notes == 'good'].shape[0]
(num_good / code_maps_pred_no_assn_complete.shape[0]) * 100

84.06593406593407

In [ ]:
code_maps_pred_no_assn_complete

In [154]:
all_corr = code_maps_correct_preds_complete[code_maps_correct_preds_complete.all_correct].shape[0]
(all_corr / code_maps_correct_preds_complete.shape[0]) * 100

96.08938547486034

In [293]:
training_data = pd.DataFrame(json.load(open('MedCAT_Export_With_Text_Assn_No_Pred.json'))['projects']).iloc[:-3]

In [297]:
sum(training_data.documents.apply(len))

3841

In [177]:
all_corr_preds_no_assn = code_maps_pred_no_assn_complete[code_maps_pred_no_assn_complete.notes == 'good']

In [188]:
all_corr_preds_corr = code_maps_correct_preds_complete[code_maps_correct_preds_complete.all_correct]

In [299]:
all_corr_preds_corr

,icd9_code,icd9_long_title,icd10_code,umls_cuis,all_correct,notes
0,845,Intestinal infection due to Clostridium difficile,['A047'],"['C1257843', 'C0494025', 'C2880102']",True,NaN
1,380,Streptococcal septicemia,['A409'],['C0152964'],True,NaN
2,389,Unspecified septicemia,['A419'],"['C2887101', 'C0036690']",True,NaN
3,42,NaN,['B20'],"['C0001857', 'C0001175', 'C0019693', 'C0864665']",True,NaN
4,7032,Chronic viral hepatitis B without mention of h...,['B181'],"['C0276610', 'C0524909', 'C2911652']",True,NaN
...,...,...,...,...,...,...
174,99592,Severe sepsis,['R6520'],"['C1719672', 'C2830469']",True,NaN
175,99681,Complications of transplanted kidney,"['T8610', 'T8611', 'T8612']","['C2891269', 'C0238217', 'C1404117']",True,NaN
176,99731,Ventilator associated pneumonia,['J95851'],['C1701940'],True,NaN
177,V08,NaN,['Z21'],"['C0019699', 'C0476550']",True,NaN


In [129]:
assn_no_pred = pd.DataFrame(json.load(open('data/MedCAT_Export_With_Text_Assn_No_Pred.json'))['projects'])

In [123]:
json.dump(preds_no_assn_json, open('data/MedCAT_Export_With_Text_Pred_No_Assn.json', 'w'))

In [253]:
preds_corr = pd.DataFrame(json.load(open('MedCAT_Export_With_Text_Pred_Corr.json'))['projects'])

In [231]:
def fix_cui_string(cui_string):
    cui_string = cui_string.replace("'", '"')
    return ','.join(json.loads(cui_string))

In [276]:
cui_strings = all_corr_preds_no_assn.umls_cuis.apply(fix_cui_string)
corr_1 = preds_no_assn[preds_no_assn.cuis.isin(cui_strings)]

In [277]:
cui_strings = all_corr_preds_corr.umls_cuis.apply(fix_cui_string)
corr_2 = preds_corr[preds_corr.cuis.isin(cui_strings)]

In [469]:
corr_concat

,name,id,cuis,tuis,documents
0,Prd N Asn - DS: 363 code: V4511 desc: Renal di...,183,"C2945570,C1399375,C2349870,C0481496",,"[{'id': 3180, 'name': '164077', 'text': ' 1. S..."
1,Prd N Asn - DS: 362 code: V441 desc: Gastrosto...,182,C0260683,,"[{'id': 3179, 'name': '176423', 'text': ' 1. S..."
2,Prd N Asn - DS: 361 code: V440 desc: Tracheost...,181,C0260682,,"[{'id': 3174, 'name': '112290', 'text': ' Obes..."
3,Prd N Asn - DS: 360 code: V1588 desc: Personal...,180,"C1268740,C2919132",,"[{'id': 3164, 'name': '160730', 'text': ' 1. C..."
4,Prd N Asn - DS: 359 code: V08 desc: Asymp hiv ...,179,"C0019699,C0476550",,"[{'id': 3154, 'name': '143757', 'text': ' stre..."
...,...,...,...,...,...
174,Cor Prd - DS: 368 code: 07032 desc: Hpt B chrn...,188,"C0276610,C0524909,C2911652",,"[{'id': 3215, 'name': '154838', 'text': ' s/p ..."
175,Cor Prd - DS: 367 code: 042 desc: Human immuno...,187,"C0001857,C0001175,C0019693,C0864665",,"[{'id': 3205, 'name': '130858', 'text': ' Mult..."
176,Cor Prd - DS: 366 code: 0389 desc: Septicemia NOS,186,"C2887101,C0036690",,"[{'id': 3195, 'name': '107520', 'text': ' Chol..."
177,Cor Prd - DS: 365 code: 0380 desc: Streptococc...,185,C0152964,,"[{'id': 3185, 'name': '184534', 'text': ' Prim..."
